In [1]:
# %tensorflow_version 2.x
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train_y = train.pop('Species')
test_y = test.pop('Species')

def input_fn(features, labels, training=True, batch_size=256):
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels)) # create tf.data.Dataset object with data and its labels
    if training:
        ds = ds.shuffle(1000).repeat() # shuffle and repeat if in training mode
    return ds.batch(batch_size)

my_feature_columns = []
for key in train.keys(): # every column name in train dataset
    my_feature_columns.append(tf.feature_column.numeric_column(key))



Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


## Building the model
- Many options for different estimators we can use, including DNNClassifier (Deep Neural Network) and LinearClassifier
    - LinearClassifier is similar to linear regression, but with probability of each classification instead of a numeric value
- Gonna use DNNClassifier because there's not necessarily a linear relationship in the data (given, not proven)

In [2]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30,10], # two hidden layers of 30 and 10 nodes respectively
    # Decided arbitrarily, we'll learn about that later
    n_classes=3 # 3 classes to choose between
)


Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.

Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.

INFO:tensorflow:Using default config.

INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\miraf\\AppData\\Local\\Temp\\tmp258cuhlh', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker',

### Training the model

In [3]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), steps=5000
) # lambda bullshit done because we just direct defined an input function instead of having the function making function

Instructions for updating:
Use tf.keras instead.


INFO:tensorflow:Calling model_fn.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.keras instead.
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\miraf\AppData\Local\Temp\tmp258cuhlh\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0

In [4]:
classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.

INFO:tensorflow:Starting evaluation at 2023-12-13T17:13:10
Instructions for updating:
Use tf.keras instead.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\miraf\AppData\Local\Temp\tmp258cuhlh\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.13927s
INFO:tensorflow:Finished evaluation at 2023-12-13-17:13:10
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, average_loss = 0.37942275, global_step = 5000, loss = 0.37942275
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\miraf\AppData\Local\Temp\tmp258cuhlh\model.ckpt-5000


{'accuracy': 0.93333334,
 'average_loss': 0.37942275,
 'loss': 0.37942275,
 'global_step': 5000}

Yay, 93% accuracy!

In [7]:
# Code to test specific datapoints
def input_fn(features, batch_size=256):
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        if not val.isdigit(): valid=False
    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))



Please type numeric values as prompted.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\miraf\AppData\Local\Temp\tmp258cuhlh\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (98.1%)
